In [1]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
# โหลดข้อมูลจากไฟล์ CSV
file_name = 'air_quality_100_years_with_quality_en.csv'
data = pd.read_csv(file_name)

In [3]:
# ตรวจสอบคอลัมน์ใน DataFrame
print(data.columns)


Index(['Date', 'PM2.5', 'PM10', 'O3', 'CO', 'AQI', 'Humidity', 'Temperature',
       'Air_Quality'],
      dtype='object')


In [4]:
# คอลัมน์ที่ต้องการตรวจสอบ (หลังจาก One-Hot Encoding)
required_columns = ['Date', 'PM2.5', 'PM10', 'O3', 'CO', 'AQI', 'Humidity', 'Temperature', 'Air_Quality']

for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Missing required column: {col}")


In [24]:
numeric_cols = X.select_dtypes(include=['number']).columns  # เลือกเฉพาะคอลัมน์ที่เป็นตัวเลข
categorical_cols = X.select_dtypes(exclude=['number']).columns  # เลือกเฉพาะคอลัมน์ที่เป็นข้อความ

# เติมค่าที่หายไปในข้อมูลตัวเลขด้วยค่าเฉลี่ย (mean)
num_imputer = SimpleImputer(strategy='mean')
X_numeric_imputed = pd.DataFrame(num_imputer.fit_transform(X[numeric_cols]), columns=numeric_cols)

# รวมข้อมูลที่เติมค่ากลับมาควบคู่กับคอลัมน์หมวดหมู่
X_imputed = pd.concat([X_numeric_imputed, X[categorical_cols]], axis=1)

# ใช้ StandardScaler ปรับสเกลข้อมูลตัวเลข
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed[numeric_cols]), columns=numeric_cols)

# แสดงข้อมูลที่ปรับสเกลแล้ว
print(X_scaled.head())

      PM2.5      PM10        O3        CO       AQI  Humidity  Temperature
0  0.124372 -0.981713  1.632656  0.557436  0.000000  0.719250     0.215560
1  1.595518 -0.726783  0.260498  0.211469  0.158648 -0.169888     1.042854
2 -0.066686  1.424190 -1.046435  0.407648 -0.982178 -0.954421    -1.818333
3 -1.556938 -0.487786  1.165259 -1.720983 -1.679349  0.039321    -0.335328
4  0.200795 -1.332242  1.392806  1.659356  1.667073 -0.954421    -1.238034


In [25]:
# แยกข้อมูลด้วย train_test_split
X_train, X_test, y_train_temp, y_test_temp = train_test_split(
    X_scaled, y_temp, train_size=0.8, random_state=42
)

# ใช้ดัชนีที่ตรงกันจาก X_train, X_test เพื่อนำมาเลือกค่าใน y_PM
#y_train_PM = y_PM.loc[X_train.index]
#y_test_PM = y_PM.loc[X_test.index]

# ตรวจสอบผล

print(y_train_temp.head())
print(y_test_temp.head())

5377     27.0
6692     96.0
4996     38.0
1561    132.0
2114     21.0
Name: PM2.5, dtype: float64
1145     24.0
8501     96.0
3903    194.0
1608     71.0
9139    130.0
Name: PM2.5, dtype: float64


In [26]:
# สร้างโมเดล RandomForest
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb.fit(X_train, y_train_temp)

y_pred_xgb = xgb.predict(X_test)
knn = KNeighborsRegressor(n_neighbors=7)

#decision_tree = DecisionTreeRegressor(random_state=42)
svm = SVR(kernel='rbf', C=5.0, epsilon=0.05)


In [28]:
xgb.fit(X_train, y_train_temp)
svm.fit(X_train, y_train_temp)
knn.fit(X_train, y_train_temp)

KNeighborsRegressor(n_neighbors=7)

In [29]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    accuracy = 100 - mape

    print(f"{model.__class__.__name__} Evaluation")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R^2 Score: {r2}")
    print(f"MAPE: {mape}%")
    print(f"Prediction Accuracy: {accuracy}%")
    print("=" * 50)
    return y_pred


In [30]:
evaluate_model(xgb, X_test, y_test_temp)
evaluate_model(svm, X_test, y_test_temp)
evaluate_model(knn, X_test, y_test_temp)

XGBRegressor Evaluation
MAE: 6.37757683290614
MSE: 359.2214566908829
R^2 Score: 0.8846111390702578
MAPE: 12.797566011829447%
Prediction Accuracy: 87.20243398817055%
SVR Evaluation
MAE: 6.8203833911793685
MSE: 354.5004111957635
R^2 Score: 0.8861276299477726
MAPE: 15.02933330180758%
Prediction Accuracy: 84.97066669819242%
KNeighborsRegressor Evaluation
MAE: 14.305692251193536
MSE: 504.0218036829128
R^2 Score: 0.8380984745552881
MAPE: 27.901324056827143%
Prediction Accuracy: 72.09867594317285%


array([ 34.14285714, 106.42857143, 176.        , ...,  32.85714286,
       101.71428571,  50.28571429])

In [31]:
# บันทึกโมเดล
joblib.dump(xgb, 'xgb_temp.pkl')
joblib.dump(xgb, 'knn_temp.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(svm, 'svm_temp.pkl')


['svm_temp.pkl']

In [32]:
# โหลดโมเดลที่เทรนไว้
model_1 = joblib.load('svm_temp.pkl')
model_2 = joblib.load('knn_temp.pkl')
model_3 = joblib.load('xgb_temp.pkl')
scaler = joblib.load('scaler.pkl')

In [34]:


# โหลดโมเดลที่เคยเทรนไว้
model_1 = joblib.load('svm_temp.pkl')  # SVM
model_2 = joblib.load('knn_temp.pkl')   # Random Forest
model_3 = joblib.load('xgb_temp.pkl')  # XGBoost

# โหลด Scaler (ถ้ามีการใช้ Normalize หรือ Standardize)
scaler = joblib.load('scaler.pkl')

# สร้าง Voting Regressor โดยใช้โมเดลที่โหลดมา
ensemble_model = VotingRegressor(estimators=[
    ('svm', model_1),
    ('knn', model_2),
    ('xgb', model_3)
])

# ฝึกสอน Voting Regressor (ถ้าต้องการ Re-train)
ensemble_model.fit(X_train, y_train_temp)

# ประเมินผลโมเดล
evaluate_model(ensemble_model, X_test, y_test_temp)

#บันทึกโมเดลที่ Train แล้ว
joblib.dump(ensemble_model, 'ensemble_model_temp.pkl')

# โหลดโมเดลที่บันทึกไว้
ensemble_model_loaded = joblib.load('ensemble_model_temp.pkl')

# ทำนายผลจากโมเดลที่โหลด
y_pred_ensemble = ensemble_model_loaded.predict(X_test)


VotingRegressor Evaluation
MAE: 6.320087066882754
MSE: 353.2805453997019
R^2 Score: 0.8865194743715199
MAPE: 13.214709807923086%
Prediction Accuracy: 86.78529019207691%
